In [1]:
%load_ext autoreload
%autoreload 2

from model import *



from torch import empty, cat, arange
from torch.nn.functional import fold, unfold
from torch import nn

In [2]:
x = torch.randn((1, 3, 4, 4))

# x = torch.tensor(
    # [[0, 1., 0],     
    # [2, 0, 0],
    # [0, 0, 1]])

# y = torch.ones(x.shape)
y = torch.ones((1, 3, 4, 4))

x, y

(tensor([[[[ 0.3076, -0.1662, -1.7081, -0.8811],
           [ 0.9332, -1.0365,  0.8057, -1.5227],
           [-0.9509,  0.1078,  0.0940, -2.0766],
           [ 2.4122, -0.1258,  0.3462,  1.3392]],
 
          [[ 1.5286, -2.5407,  1.6176, -0.1127],
           [-0.4696, -0.7762, -0.4973,  1.9874],
           [ 1.0639, -0.2287,  0.1522, -0.9330],
           [-0.0557,  1.3332, -0.9440,  0.3977]],
 
          [[ 1.7426,  0.8656, -0.6888, -0.9325],
           [ 0.1123, -0.7247,  0.7746, -0.3012],
           [-0.1316, -0.3452, -0.3065, -0.0761],
           [-1.7047, -2.0099, -0.7010,  0.0794]]]]),
 tensor([[[[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]]]]))

In [94]:
conv = nn.Conv2d(3, 4, 2)
expected = conv(y)
expected

tensor([[[[ 0.6936,  0.6936,  0.6936],
          [ 0.6936,  0.6936,  0.6936],
          [ 0.6936,  0.6936,  0.6936]],

         [[ 0.1451,  0.1451,  0.1451],
          [ 0.1451,  0.1451,  0.1451],
          [ 0.1451,  0.1451,  0.1451]],

         [[-0.7132, -0.7132, -0.7132],
          [-0.7132, -0.7132, -0.7132],
          [-0.7132, -0.7132, -0.7132]],

         [[ 0.7310,  0.7310,  0.7310],
          [ 0.7310,  0.7310,  0.7310],
          [ 0.7310,  0.7310,  0.7310]]]])

In [96]:
# torch.nn.Unfold(kernel_size=2)(x)

kernel_size = (2, 2)
out_channels = 4

# Output of convolution as a matrix product
unfolded = torch.nn.functional.unfold(y, kernel_size=kernel_size)
# unfolded

wxb = conv.weight.view(out_channels, -1) @ unfolded + conv.bias.view(1, -1, 1)
actual = wxb.view(1, out_channels, y.shape[2] - kernel_size[0] + 1, y.shape[3] - kernel_size[1]+ 1)
torch.testing.assert_allclose(actual, expected)

### Testing our Conv2d Layer 

Forward pass

In [133]:
# testing our conv2d layer
my_conv = Conv2d(3, 4, 2)
x2 = my_conv.forward(x)
x2

tensor([[[[-0.1934,  0.6827, -1.0966],
          [ 0.0961,  0.2508,  0.3367],
          [-0.3127,  0.6376, -0.2396]],

         [[-0.4834, -0.1858, -0.7048],
          [-0.0870, -0.5169, -0.5492],
          [-0.6541, -0.4518, -0.8561]],

         [[ 0.3024, -0.9491,  0.3033],
          [ 0.1651, -0.5372,  0.0725],
          [ 0.4081, -0.0865,  0.3034]],

         [[ 0.6818, -0.6914,  1.0980],
          [-0.7440,  0.0023, -0.6106],
          [ 0.5955, -1.0637,  0.2999]]]])

In [61]:
torch.testing.assert_allclose(my_conv.forward(x), torch.nn.functional.conv2d(x, my_conv.weight, my_conv.bias))

Backward pass

### Other tests

In [5]:
conv.weight.view(out_channels, -1).size()

torch.Size([4, 12])

In [118]:
x.size()

torch.Size([1, 3, 4, 4])

In [112]:
unfolded.size()

torch.Size([1, 12, 9])

In [115]:

wxb.size()

torch.Size([1, 4, 9])

In [81]:
# relu = ReLU()
f=Sigmoid()

f.forward(x)

tensor([[0.5000, 0.7311, 0.5000],
        [0.8808, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.7311]])

In [82]:
f.backward(y)

tensor([[0.5000, 0.2689, 0.5000],
        [0.1192, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.2689]])